In [1]:
####################################################################################################################
#MLDash
#Developed by Lukas Mueller (2023)
#CS-499 Capstone Project
#BS in Computer Science(conc. in Software Engineering)
####################################################################################################################

# Welcome to the MLDash project, which was written in the Spring of '23. As an aspiring machine learning engineer, 
#and software engineer by discipline, I recognize the value in making complicated tasks more accessible to the common
#user. Training machine learning algorithms requires tuning their (sometimes numerous) hyperparameters and hoping
#that the changes made boost model performance, while also keeping track of the changes that have been made.
#The main purpose of this project is to simplify this task for users, automate record keeping, and generate data
#visualizations that can assist users in the selection of more optimal hyperparameters. While the project currently
#hosts only one AI algorithm, OpenAI Gym's Cartpole-v1, I anticipate that additional models can be added as modules
#over time. The cartpole algorithm is deep-Q reinforcement learning algorithm that solves the problem of balancing
#a pole attached to cart that moves on a frictionless rail. The model acquires data from the environment
#(observation space) and determines which sequence of actions results in better performance for each situation. 
#The model trains itself by replaying experiences it acquires, prioritizing samples of its experience based on how 
#different those experiences were from its predictions. The model begins training with a higher likelyhood to explore,
#allowing it to gather new experiences by trying numerous different action sequences; as training continues, the model
#relies less on exploration and more on the exploitation of the experiences it has gathered. 

#If you are reading this, you are looking at the central file for the project. This file creates an instance of the 
#MLMongo class (defined in crud.py) to initiate a connection to a local MongoDB service. 
#It also creates an instance of the Cartpole class, which makes use of the DQNsolver(for the learning model setup, and 
#learning procedures), PrioritizedBuffer(an upgraded sampling procedure that makes experience replay a non arbitrary 
#process), and ScoreLogger(used to record hyperparameter profiles along with key performance metrics) classes. During 
#training, hyperparameter values and performance metrics are written to the metrics.csv file. The metrics 
#collection and summary collection are contained in the MongoDB TRAIN database. When the model successfully balances 
#the pole (for a minimum average duration) the metrics.csv file that contains training data is read into a Python 
#dictionary and written to the TRAIN/metrics database collection. Once the database has been populated with training 
#data, users can view this data using the dashboard. The dashboard allows the data to be viewed in the form of an 
#interactive data table and dynamic data visualizations. Users have the ability to filter data to examine summary 
#data and individual training sessions, and the accompanying charts change along with the datatable.

#This projec was created by integrating a simpler dashboard program and a basic cartpole learning model implementation.
#Various upgrades were made, along the way, in areas related to software engineering, algorithms and data structures,
#and data structures:

#The names of files, functions, and variables were updated to reflect the new context

#Dependencies were updated to reflect more recent versioning

#Significant amounts of documentation was added in the form of comments and a readme file.

#Code was integrated and refactored into a more modular form that relies on seperate classes.

#Additional functions were added to the crud.py, score_logger.py to facilitate local storage of learning model training
#data, and the ability to write this data to a database. The Dashboard.ipynb file was upgraded to provide a relevant 
#and task-specific interface with enhanced interactivity, dataframe filtering, data session indexing, and data 
#visualization capabilities.

#The learning algorithm itself enjoyed a significant upgrade that improved its ability to learn from its experiences.
#During exprience replay (code found in dqn_solver.py) the batch sampling method was upgraded from using a random
#sampling procedure to use prioritized sampling (code found in the prioritized_buffer.py file), resulting in a more
#effective implementation of the learning algorithm and contributing to significant reduction in total memory use. 

#By using tool that helps cultivate data perspective and insights, the process of training a deep-Q reinforcement
#learning algorithm becomes more intuitive for users, while reducing the cognitive burden associated with analyzing
#huge amounts of data. Enjoy!
###################################################################################################################

In [2]:
#Dashboard imports

from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State
import os
import numpy as np
import pandas as pd
from bson.json_util import dumps
from bson.objectid import ObjectId
import base64
from crud import MLMongo
from cartpole import MLCartpole
import statsmodels
import csv
from security import Security


# Tensorflow will automatically detect CUDA enabled GPUs on the host system, with the amount displayed to the user
#(Requires compatible hardware and additional setup. Refer to the README document for further details)
import tensorflow as tf
print("# of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#########################
# CONSTANTS (file references)
#########################
METRICS_FILE = "metrics.csv"
SUMMARY_FILE = "summary.csv"
IMAGE_FILE = 'logo.png'   
#reads a local image file and encodes it as a base64 string. 
#The open() function is used to open the file in binary mode, 
#and base64.b64encode() is used to encode the binary data as a base64 string.
#(displays images embedded in the app layout using the html.Img() component)
encoded_image = base64.b64encode(open(IMAGE_FILE, 'rb').read())

############################
# Globals
############################
session_count = 0
animation_enabled = True


#########################
# Session count
#########################
# Helper functions to help keep track of distinct training data by session and facilitate the display the most
# recently completed session number                              
def count_sessions():
    global session_count
    dFrame = pd.DataFrame.from_records(mongobject.read_all({}))
    dFrame = dFrame.iloc[:,0:]
    try:
        df_sorted = dFrame.sort_values(by='session', ascending=False)
        top_session = df_sorted.head(1)['session'] 
        session_count = top_session.item()
        if session_count == None:
            session = 0
    except:
        session_count = 0

    return int(session_count)
     

def increment_session():
    global session_count
    session_count = count_sessions() + 1 
        
    return session_count


#############################
# Data Manipulation / Model
#############################
#False setup (will not authenticate), in order to set up gloabal variables for later use
#creates a security object, with every such instance using a randomized key
secure = Security()
#gets XOR encoded versions of the username and password
xor_name, xor_pass = secure.xor_encode(" ", " ")
#creates a database object, which decodes the encoded credentials using the same security object
mongobject = MLMongo(secure, xor_name, xor_pass)


                               
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('__name__')

##############################
#Dashboard Login
##############################
# Define the login form as a separate function
def login_form():
    return html.Div([
        dcc.Input(id='username-input', type='text', placeholder='Username'),
        dcc.Input(id='password-input', type='password', placeholder='Password'),
        html.Button('Login', id='login-button'),
        html.Div(id='login-error')   
    ])

##############################
#Dashboard Main content
##############################
# Define the main content as a separate function
def main_content():
    global mongobject
    global df
    
    return html.Div([
        #Main content
            html.Div( #Outer div number 1
                children=[
                    html.Div([
                        html.Div(id='hidden-div', style={'display':'none'}),
                        html.Center(html.B(html.H1('Machine Learning Dashboard'))),
                        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
                        html.Center(html.B(html.H2("Developed by Lukas Mueller (2023)"))),
                        html.Hr() 
                   ])
                ]),


            html.Div([ #Outer div number 2
                html.Div(className='row',
                    style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
                        children=[
                            html.Div(),
                            html.Div(
                                html.Label('Alpha:'),
                                className='col s12 m6'
                                ),
                            html.Div(
                                dcc.Input(id='input-1', type='number', value= 0.01, placeholder="0>x>1, low"),
                                className='col s12 m6'
                                ),
                            html.Div(style={'text-align': 'right', 'margin-right': '10px'},
                                    children = [
                                        html.Div(id='reduce_to_rubble'),
                                        html.Button('Wipe DB', id = 'wipe-database-button', n_clicks = 0)
                                    ]
                                    )
                        ]
                        ),

                html.Div(className='row',
                    style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
                        children=[
                            html.Div(),
                            html.Div(
                                html.Label('Gamma:'),
                                className='col s12 m6'
                            ),
                            html.Div(
                                dcc.Input(id='input-2', type='number', value= 0.90, placeholder="0>x>1, high"),
                                className='col s12 m6'
                                ),
                            html.Div()
                        ]
                        ),

                html.Div(className='row',
                    style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
                        children=[
                            html.Div(),
                            html.Div(
                                html.Label('Epsilon Min:'),
                                className='col s12 m6'
                            ),
                            html.Div(
                                dcc.Input(id='input-3', type='number', value= 0.01, placeholder="0>x>1, low"),
                                className='col s12 m6'
                            ),
                            html.Div()
                        ]
                        ),

                html.Div(className='row',
                    style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
                        children=[
                            html.Div(),
                            html.Div(
                                html.Label('Epsilon Max:   '),
                                className='col s12 m6'
                            ),
                            html.Div(
                                dcc.Input(id='input-4', type='number',  value=0.5, placeholder="0>x>1, high"),#alt value=0.75
                                className='col s12 m6'
                            ),
                            html.Div()
                        ]
                        ),

                html.Div(className='row',
                    style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
                        children=[
                            html.Div(),
                            html.Div(
                                html.Label('Epsilon Decay: '),
                                className='col s12 m6'
                            ),
                            html.Div(
                                dcc.Input(id='input-5', type='number', value= 0.925, placeholder="0>x>1, high"),#alt value=0.95 
                                className='col s12 m6'
                                ),
                            html.Div()
                        ]
                        ),

                html.Div(className='row',
                    style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
                        children=[
                            html.Div(),
                            html.Div(
                                html.Label('# of Episodes: '),
                                className='col s12 m6'
                                ),
                            html.Div(
                                dcc.Input(id='input-6', type='number', value= 20, placeholder="10-40"),
                                className='col s12 m6'
                                ),
                            html.Div()
                        ]
                        ),

                html.Div(className='row',
                    style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
                        children=[
                            html.Div(),
                            html.Div(
                                html.Label('Batch Size: '),
                                className='col s12 m6'
                                ),
                            html.Div(
                                dcc.Input(id='input-7', type='number', value=40, placeholder="Default value: 20"),#alt value=20
                                className='col s12 m6'
                                ),

                            html.Div()
                        ]
                        ),

                html.Div(className='row',
                    style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
                        children=[
                            html.Div(),
                            html.Div(
                                html.Label('Buffer Size: '),
                                className='col s12 m6'
                                ),
                            html.Div(
                                dcc.Input(id='input-8', type='number', value=12000, placeholder="Default value: 12000"),
                                className='col s12 m6'
                                ),

                            html.Div(
                                html.Label("Latest Training Session #: ")
                            )
                        ]
                        ),


                html.Div(className='row',
                    style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
                        children=[
                            html.Div(),
                            html.Div(),
                            html.Div(style={'text-align': 'center', 'margin-top': '10px', 'margin-right': '10px'},
                            # The submit button saves the hyperparameter values the user enters and uses them to create an instance of the 
                            # Cartpole class. This initiates a reinforcement learning training session.
                                children = [
                                    html.Div(id='output'),
                                    html.Button('Submit', id='submit-button', n_clicks=0)
                                ]),
                            html.Div(
                                html.Label(id="session-total",
                                children = str(session_count)
                                )
                            )
                        ]
                        )
            ]),
        ###############################################################
        # Radio Buttons
        ###############################################################

                html.Hr(),
                html.Br(),

                #this row houses four radio buttons that are used to filter the data
                html.Div(className='row', #Outer div number 3
                    style={'display': 'flex',
                         'justify-content': 'space-between'  # Add justify-content property
                           },
                        children=[
                            dcc.RadioItems(
                                id="filter-type",
                                #Labels are provisioned for useful filtering of performance metrics
                                options=[
                                    {'label': 'High Scores', 'value': 'highscore'},
                                    {'label': 'Low Runs', 'value': 'lowruns'},
                                    {'label': 'Both', 'value': 'both'},
                                    {'label': 'All', 'value': 'all'}
                                ],
                                value='all'
                            ),
                            # Toggle cartpole animation on or off
                            dcc.RadioItems(
                                id="py-game",
                                options=[
                                    {'label': 'Animation Enabled', 'value': "enabled"},
                                    {'label': 'Animation Disabled', 'value': "disabled"}
                                ],
                                value='enabled'
                            )
                        ]
                ),

                html.Hr(),
        
        ########################################################
        ####### This is the layout for the data table
        #######################################################

                dt.DataTable(
                    id='datatable-id',
                    columns=[
                        {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                    ],

                    #these are options which mostly provide for native interactivity with the table
                    data=df.to_dict('records'),
                    editable= False,
                    filter_action="native",
                    sort_action="native",
                    sort_mode="multi",
                    column_selectable= False,
                    row_selectable= False,
                    selected_columns=[],
                    selected_rows=[],
                    page_action="native",
                    page_current=0,
                    page_size=200,
                    style_table={'overflowX': 'auto', 'overflowY': 'visible', 'maxHeight': '500px', 'maxWidth': '100%', 'minWidth': '100%', 'position': 'relative'},
                    

                    #Below is some simple styling for the data table to make it easier to look at...

                    #table striping implemented to make visual tracking of a document's data easier
                    style_cell_conditional=[
                        {
                            'if': {'column_id': c},
                            'textAlign': 'left'
                        } for c in ['Date', 'Region']
                    ],
                    style_data={
                        'color': 'black',
                        'backgroundColor': 'white'
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(220, 220, 220)',
                        },
                        {
                            'if': {'row_index': 'even'},
                            'backgroundColor': 'rgb(240, 240, 240)',
                        }
                    ],
                    style_header={
                        'backgroundColor': 'rgb(210, 210, 210)',
                        'color': 'black',
                        'fontWeight': 'bold'
                    },
                    #sizing for table cells, to increase presentability based on length of data
                    style_cell={
                        'minHeight': '16px', 'height': '16px', 'maxHeight': '16px',
                        'minWidth': '160px', 'width': '160px', 'maxWidth': '160px',
                        'whiteSpace': 'normal', 'textAlign': 'left', 'position': 'sticky', 'top': '0'
                        
                    }),

                html.Br(),
                html.Hr(),
        
        ###########################################################################
        ####### This sets up the dashboard so that two charts are diplayed per row
        ###########################################################################
                # Row 1 (all sessions)
                html.Div(className='row',
                        style={'display' : 'flex'},
                            children=[
                            #this is the first chart: pie chart (correlation between session and total runs)
                            html.Div(
                                id='graph1-id',
                                className='col s12 m6',
                                ),
                            #this is the second chart: bar chart  (correlation between session and total runs)
                            html.Div(
                                id='graph2-id',
                                className='col s12 m6',
                                )]),

                # Row 2 (filter-specific)
                html.Div(className='row',
                        style={'display' : 'flex'},
                            children=[
                            #this is the third chart: scatterplot with linear regression (correlation between runs and max score)
                            html.Div(
                                id='graph3-id',
                                className='col s12 m6',
                                ),
                            #this is the second chart: scatterplot with lowess trendline (correlation between runs and max score)
                            html.Div(
                                id='graph4-id',
                                className='col s12 m6',
                                )]),

                #Row 3 (filter-specific)
                html.Div(className='row',
                        style={'display' : 'flex'},
                            children=[
                            #this is the first chart: scatterplot with linear regression (correlation between runs and exploration rate)
                            html.Div(
                                id='graph5-id',
                                className='col s12 m6',
                                ),
                            #this is the second chart: scatterplot with lowess trendline (correlation between runs exploration rate)
                            html.Div(
                                id='graph6-id',
                                className='col s12 m6',
                                )]),

                #Row 4 (filter-specific)
                html.Div(className='row',
                    style={'display' : 'flex'},
                        children=[
                            #this is the first chart: scatterplot with linear regression (correlation between exploration rate and max score)
                            html.Div(
                                id='graph7-id',
                                className='col s12 m6',
                                ),
                            #this is the second chart: scatterplot with lowess trendline (correlation between exploration rate and max score)
                            html.Div(
                                id='graph8-id',
                                className='col s12 m6',
                            )
                        ])
        ])


# The login page is displayed first. Only after the user is authenticated does the rest of the dashboard appear.
app.layout = login_form() 


####################################
# Login Screen Callback
####################################
 
# Define a login screen using the dcc.Input component and dash.dependencies.State decorator
@app.callback(dash.dependencies.Output('login-error', 'children'),
              [dash.dependencies.Input('login-button', 'n_clicks')],
              [dash.dependencies.State('username-input', 'value'),
               dash.dependencies.State('password-input', 'value')])

def authenticate(n_clicks, username, password):
    global mongobject
    global secure
    global df
    
    if n_clicks is None:  # Button has not been clicked yet
        return ""
    
    elif n_clicks > 0:
        authenticated = False
        #creates a security object, with every sucj instance using a randomized key
        secure = Security()
        #gets XOR encoded versions of the username and password
        xor_name, xor_pass = secure.xor_encode(username, password)
        #creates a database object, which decodes the encoded credentials using the same security object
        mongobject = MLMongo(secure, xor_name, xor_pass)
        
        # check if authentication was successful
        authenticated = mongobject.is_authenticated()
        
        if authenticated == True:
            df = pd.DataFrame.from_records(mongobject.read_all({}))
            df = df.iloc[:,0:]
            # Show the main content if the authentication is successful. The layout displays a logo and title, followed by a grid layout that accepts user input in multiple ways
            # Once the database is populated, the input section is followed by an datatable
            # Beneath the datatable, data is visualized using several charts, which are dynamically updated to reflect visible datatable data
            return main_content()
        
        
        else:
            print("There is something wrong with the authentication mechanism. Please contact the Developer.")
           
    else:
        return "There is something wrong with your interface or callback function"
    
    

    
#############################################
# Interaction Between Components / Controller
#############################################

#This function can be used to store user input values for hyperparameter to local variables, but in this case they are used ("on-the-fly") 
# as function arguments. This causes the cartpole function execute as soon as the user submits their input data. 
@app.callback(Output('output', 'children'),
              [Input('submit-button', 'n_clicks')], 
              [State('input-1', 'value'),
               State('input-2', 'value'),
               State('input-3', 'value'),
               State('input-4', 'value'),
               State('input-5', 'value'),
               State('input-6', 'value'),
               State('input-7', 'value'),
               State('input-8', 'value')])      
   
def update_output(n_clicks, input1, input2, input3, input4, input5, input6, input7, input8):  
    
    global session_count
    global animation_enabled
    solved = False
    
    fields_metrics = ['session', 'run', 'meanScores', 'maxScores', 'globalMax', 'alpha', 'gamma', 'exploreRate', 'batchSize', 'bufferSize']
    fields_summary = ['session', 'totalRuns', 'totalSteps', 'globalMax', 'alpha', 'gamma', 'epsilonMin', 'epsilonMax', 'epsilonDecay', 'episodes', 'batchSize', 'bufferSize']
    
    if n_clicks > 0:
        print("Please wait while the learning algorithm trains. Performance metrics will then be viewable.") 
        
        #increment session count to tag this session with a session id equal to stored sessions + 1
        session_count = increment_session()
        
        # This is the line of code that starts the reinforcement learning algorithm 
        cartpole_instance = MLCartpole(session_count, animation_enabled, input1, input2, input3, input4, input5, input6, input7, input8) 
        
        # If the algorithm was successful, solved is assigned the value of True
        solved = cartpole_instance.cartpole(cartpole_instance.cart_animation)
        
        
        if solved == True:
            print("This model was successful. Congratulations! Writing to database...")
             
            try:
                mongobject.import_csv(METRICS_FILE, fields_metrics)
               
            except:
                print("There was an issue preventing records from being written to the database. Please contact your IT Administrator, local developer, and nearest coffee house.")
            
            try:
                mongobject.import_csv(SUMMARY_FILE, fields_summary)
                
            except:
                print("There was an unknown issue writing summary data to the database.")
                
            update_dashboard('reset')
            return f'Dashboard updated: {solved}'
        
        else:
            print("This model instance exceeded the run limit. Enter a different set of values, and try again")
            return ''  



# This function deletes all data stored in the metrics and summary collections in the TRAIN database.
# It used another function defined in crud.py, but has been repackaged here to occur when the user clicks on the "Wipe DB" button
@app.callback(Output('reduce_to_rubble', 'children'),
              [Input('wipe-database-button', 'n_clicks')])

def wipe_stored_data(n_clicks):
    if n_clicks > 0:
        mongobject.clear_collections()    
    
    
#Radio filter callback, with different options to filter for learning algorithm params. 
#Reset filter removes other filters to display all data.
@app.callback([Output('datatable-id','data'), 
               Output('datatable-id','columns'), 
               Output('session-total', 'children')],
              [dash.dependencies.Input('filter-type', 'value')])

def update_dashboard(selected_value):
    global session_count
    df = pd.DataFrame.from_records(mongobject.read_all({}))
    df = df.iloc[:,0:]
    
    # Update the displayed session count
    session_count = count_sessions()
        
    # Construct the new label text
    updated_session_count = html.Label(id="session-total", children=str(session_count))

    #filters data to only display runs that scored above 100 points
    if selected_value == 'highscore':
        df = pd.DataFrame.from_records(mongobject.read_all({
            "session" :  {"$gte":0, "$lte":session_count},
            "run" : {"$gte":0, "$lte":3000},
            "maxScores": {"$gte":100, "$lte":1000}
        }))
        

     #filters data to only display sessions where the model solved the problem in 2000 runs or less
    elif selected_value == 'lowruns':
        df = pd.DataFrame.from_records(mongobject.read_all({
            "session" :  {"$gte":0, "$lte":session_count},
            "run": {"$not": {"$elemMatch": {"$gt": 2000}}},
            "maxScores": {"$gte":0, "$lte":1000}
        }))
        

    #filters data to only show high scores from training sessions that solved the problem in 2000 runs or less
    elif selected_value == 'both':
        df = pd.DataFrame.from_records(mongobject.read_all({
            "session" : {"$gte":0, "$lte":session_count},
            "run": {"$not": {"$elemMatch": {"$gt": 2000}}},
            "maxScores": {"$gte":100, "$lte":1000}
        }))
        

    # Reset query filter by querying all the data without specifying any particulars
    elif selected_value == 'all':
        df = pd.DataFrame.from_records(mongobject.read_all({}))
        
        
    
    # store the columns and the queried data in the columns and data variable, and return to enable display
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    df = df.iloc[:,0:]
    data=df.to_dict('records')
    
    return data,columns,[updated_session_count]


#Highlight any column that is selected via checkboxes displayed in header cells
@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')])

def update_style_c(selected_columns):
    return [{'if': { 'column_id': i }, 'background_color': '#D2F3FF'} for i in selected_columns]


####################################
# Enable/Disable pygame animation
####################################
@app.callback(Output('dummy', 'children'),
              [Input('py-game', 'value')])

def update_output(cart_anim):
    global animation_enabled
    
    if input0 == "enabled":
        animation_enabled = True

    elif input0 == "disabled":
        animation_enabled =  False

    else:
        animation_enabled = True
        print("There is an issue with animation logic. Cannot disable animation. Please contact the Developer.")
       
    return dash.no_update
####################################
# Update all charts
####################################

#These first two charts display summary data from all data in the dataframe, making them less responsive to filtering
# Updates the first chart
@app.callback(Output('graph1-id', "children"),
              [Input('datatable-id', "derived_viewport_data")])


def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #1."
    
    viewData = {'session': [row['session'] for row in viewData]}
    
    dff = pd.DataFrame.from_dict(viewData)    
    fig = px.pie(dff, names ='session') 
    pieChart = [dcc.Graph(figure=fig)]
    
    return pieChart


# Updates the second chart
@app.callback(Output('graph2-id', "children"),
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #2."
    
    viewData = {'session': df['session'].tolist(),'run': df['run'].tolist()}
    
    dff = pd.DataFrame.from_dict(viewData) 
    fig = px.bar(dff, x='session', y='run', labels={'x_column': 'session', 'y_column': 'run'})
    barChart = [dcc.Graph(figure=fig)]
    return barChart


#The rest of the charts change to show data that is in the current data table, only
#This makes them responsive to filtering and search functionality


# Updates the third chart (run vs max score)
@app.callback(Output('graph3-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #3."

    
    # Convert viewData dictionary to a list of rows, skipping the first rows (header)
    rows = [[row['run'], row['maxScores']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['run', 'maxScores'])
    
    
    fig = px.scatter(dff, x='run', y='maxScores', labels={'x_column': 'Runs', 'y_column': 'Maximum Score'})
    
    
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot


# Updates the fourth chart (run vs max score, with trendline)
@app.callback(Output('graph4-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #4."

    
    # Convert viewData dictionary to a list of rows, skipping the first rows (header)
    rows = [[row['run'], row['maxScores']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['run', 'maxScores'])
    
    # Filter out rows with zero 'run' values
    dff_filtered = dff[dff['run'] != 0]
    
    #drop rows with NaN values
    dff_filtered.dropna()
    
    fig = px.scatter(dff_filtered, x='run', y='maxScores', labels={'x_column': 'Runs', 'y_column': 'Maximum Score'}, trendline="ols")
    
    
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot


# Updates the fifth chart (run vs explore rate)
@app.callback(Output('graph5-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #5."

    
     # Convert viewData dictionary to a list of rows, skipping the first row (header)
    rows = [[row['run'], row['exploreRate']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['run', 'exploreRate'])

    
    fig = px.scatter(dff, x='run', y='exploreRate', labels={'x_column': 'Runs', 'y_column': 'Experience Based/Exploration'})
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot


# Updates the sixth chart (run vs explore rate, with trendline)
@app.callback(Output('graph6-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #6."

    
     # Convert viewData dictionary to a list of rows, skipping the first row (header)
    rows = [[row['run'], row['exploreRate']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['run', 'exploreRate'])
  
    # Filter out rows with zero 'run' values
    dff_filtered = dff[dff['run'] != 0]
    
    #drop rows with NaN values
    dff_filtered.dropna()
    
    fig = px.scatter(dff_filtered, x='run', y='exploreRate', labels={'x_column': 'Runs', 'y_column': 'Experience Based/Exploration'}, trendline ="ols")
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot


# Updates the seventh chart (exploration vs maximum score)
@app.callback(Output('graph7-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #7."

    
     # Convert viewData dictionary to a list of rows, skipping the first row (header)
    rows = [[row['exploreRate'], row['maxScores']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['exploreRate', 'maxScores'])
    

    fig = px.scatter(dff, x='exploreRate', y='maxScores', labels={'x_column': 'Experience Based/Exploration', 'y_column': 'Maximum Score'})
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot


# Updates the eighth chart (exploration vs maximum score with trendline)
@app.callback(Output('graph8-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #8."
    
    
     # Convert viewData dictionary to a list of rows, skipping the first row (header)
    rows = [[row['exploreRate'], row['maxScores']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['exploreRate', 'maxScores'])

    # Filter out rows with zero 'run' values
    dff_filtered = dff[dff['exploreRate'] != 0.0]
    
    #drop rows with NaN values
    dff_filtered.dropna()
    
    fig = px.scatter(dff_filtered, x='exploreRate', y='maxScores', labels={'x_column': 'Experience Based/Exploration', 'y_column': 'Maximum Score'}, trendline = "ols")
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot


# This displays the app content in Jupyter Notebook; A link appears in the notebook that allows the user to opt for 
# a full-screen view after execution
# of the application in their default web browser (http://localhost:8890)
if __name__ == '__main__':
    app.run_server(mode='inline', port=8890)

# of GPUs Available:  0
Dash is running on http://127.0.0.1:8890/



Please wait while the learning algorithm trains. Performance metrics will then be viewable.
Animation Enabled: True


C:\Users\Singularus\anaconda3\envs\ml37\lib\site-packages\gym\core.py:318: DeprecationWarning:

WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.

C:\Users\Singularus\anaconda3\envs\ml37\lib\site-packages\gym\wrappers\step_api_compatibility.py:40: DeprecationWarning:

WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.



1/1 [==============================] - 0s 33ms/step
Run: 1, exploration percent: 70.0, score: 33
Scores: (Minimum Score: 33 MovingAvg: 33, MaxAvg: 33, LocalMaxScore: 33, GlobalMaxScore: 33)

1/1 [==============================] - 0s 34ms/step
Run: 2, exploration percent: 70.0, score: 19
Scores: (Minimum Score: 19 MovingAvg: 26, MaxAvg: 33, LocalMaxScore: 33, GlobalMaxScore: 33)

1/1 [==============================] - 0s 33ms/step
Run: 3, exploration percent: 70.0, score: 15
Scores: (Minimum Score: 15 MovingAvg: 22.333333333333332, MaxAvg: 33, LocalMaxScore: 33, GlobalMaxScore: 33)

1/1 [==============================] - 0s 33ms/step
Run: 4, exploration percent: 70.0, score: 27
Scores: (Minimum Score: 15 MovingAvg: 23.5, MaxAvg: 33, LocalMaxScore: 33, GlobalMaxScore: 33)

1/1 [==============================] - 0s 32ms/step
Run: 5, exploration percent: 70.0, score: 24
Scores: (Minimum Score: 15 MovingAvg: 23.6, MaxAvg: 33, LocalMaxScore: 33, GlobalMaxScore: 33)

1/1 [====================

1/1 [==============================] - 0s 20ms/step
Run: 18, exploration percent: 70.0, score: 9
Scores: (Minimum Score: 8 MovingAvg: 19.5, MaxAvg: 33, LocalMaxScore: 39, GlobalMaxScore: 39)

1/1 [==============================] - 0s 17ms/step
Run: 19, exploration percent: 70.0, score: 40
Scores: (Minimum Score: 8 MovingAvg: 20.57894736842105, MaxAvg: 33, LocalMaxScore: 40, GlobalMaxScore: 40)

1/1 [==============================] - 0s 16ms/step
Run: 20, exploration percent: 70.0, score: 13
Scores: (Minimum Score: 8 MovingAvg: 20.2, MaxAvg: 33, LocalMaxScore: 40, GlobalMaxScore: 40)

1/1 [==============================] - 0s 18ms/step
Run: 21, exploration percent: 70.0, score: 14
Scores: (Minimum Score: 8 MovingAvg: 19.904761904761905, MaxAvg: 33, LocalMaxScore: 40, GlobalMaxScore: 40)

1/1 [==============================] - 0s 17ms/step
Run: 22, exploration percent: 70.0, score: 13
Scores: (Minimum Score: 8 MovingAvg: 19.59090909090909, MaxAvg: 33, LocalMaxScore: 40, GlobalMaxScore: 4

...

1/1 [==============================] - 0s 16ms/step
Run: 544, exploration percent: 6.893396315285283, score: 12
Scores: (Minimum Score: 8 MovingAvg: 14.4, MaxAvg: 33, LocalMaxScore: 23, GlobalMaxScore: 56)

1/1 [==============================] - 0s 15ms/step
Run: 545, exploration percent: 6.893396315285283, score: 13
Scores: (Minimum Score: 8 MovingAvg: 14.475, MaxAvg: 33, LocalMaxScore: 23, GlobalMaxScore: 56)

1/1 [==============================] - 0s 16ms/step
Run: 546, exploration percent: 6.893396315285283, score: 12
Scores: (Minimum Score: 8 MovingAvg: 14.5, MaxAvg: 33, LocalMaxScore: 23, GlobalMaxScore: 56)

1/1 [==============================] - 0s 16ms/step
Run: 547, exploration percent: 6.893396315285283, score: 10
Scores: (Minimum Score: 8 MovingAvg: 14.4, MaxAvg: 33, LocalMaxScore: 23, GlobalMaxScore: 56)

1/1 [==============================] - 0s 19ms/step
Run: 548, exploration percent: 6.893396315285283, score: 13
Scores: (Minimum Score: 8 MovingAvg: 14.45, MaxAvg: 33, Lo

1/1 [==============================] - 0s 16ms/step
Run: 555, exploration percent: 6.893396315285283, score: 13
Scores: (Minimum Score: 9 MovingAvg: 14.775, MaxAvg: 33, LocalMaxScore: 23, GlobalMaxScore: 56)

1/1 [==============================] - 0s 15ms/step
Run: 556, exploration percent: 6.893396315285283, score: 11
Scores: (Minimum Score: 9 MovingAvg: 14.825, MaxAvg: 33, LocalMaxScore: 23, GlobalMaxScore: 56)

1/1 [==============================] - 0s 17ms/step
Run: 557, exploration percent: 6.893396315285283, score: 9
Scores: (Minimum Score: 9 MovingAvg: 14.775, MaxAvg: 33, LocalMaxScore: 23, GlobalMaxScore: 56)

1/1 [==============================] - 0s 16ms/step
Run: 558, exploration percent: 6.893396315285283, score: 12
Scores: (Minimum Score: 9 MovingAvg: 14.825, MaxAvg: 33, LocalMaxScore: 23, GlobalMaxScore: 56)

1/1 [==============================] - 0s 18ms/step
Run: 559, exploration percent: 6.893396315285283, score: 10
Scores: (Minimum Score: 9 MovingAvg: 14.75, MaxAvg: 3

1/1 [==============================] - 0s 17ms/step
Run: 561, exploration percent: 6.204056683756755, score: 40
Scores: (Minimum Score: 9 MovingAvg: 15.125, MaxAvg: 33, LocalMaxScore: 40, GlobalMaxScore: 56)

1/1 [==============================] - 0s 18ms/step
Run: 562, exploration percent: 6.204056683756755, score: 47
Scores: (Minimum Score: 9 MovingAvg: 15.875, MaxAvg: 33, LocalMaxScore: 47, GlobalMaxScore: 56)

1/1 [==============================] - 0s 17ms/step
Run: 563, exploration percent: 6.204056683756755, score: 61
Scores: (Minimum Score: 9 MovingAvg: 16.875, MaxAvg: 33, LocalMaxScore: 61, GlobalMaxScore: 61)

1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 17ms/step
Run: 564, exploration percent: 6.204056683756755, score: 37
Scores: (Minimum Score: 9 MovingAvg: 17.35, MaxAvg: 33, LocalMaxScore: 61, GlobalMaxScore: 61)

1/1 [==============================] - 0s 16ms/step
Run: 565, exploration percent: 6.204056683756755, score: 69
Scores: (Minimum Score: 9 MovingAvg: 18.6, MaxAvg: 33, LocalMaxScore: 69, GlobalMaxScore: 69)

1/1 [==============================] - 0s 17ms/step
Run: 566, exploration percent: 6.204056683756755, score: 44
Scores: (Minimum Score: 9 MovingAvg: 19.3, MaxAvg: 33, LocalMaxScore: 69, GlobalMaxScore: 69)

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step
Run: 567, exploration percent: 6.204056683756755, score: 44
Scores: (Minimum Score: 9 MovingAvg: 19.925, MaxAvg: 33, LocalMaxScore: 69, GlobalMaxScore: 69)

1/1 [==============================] - 0s 16ms/step
Run: 568, exploration percent: 6.204056683756755, score: 57
Scores: (Minimum Score: 9 MovingAvg: 21, MaxAvg: 33, LocalMaxScore: 69, GlobalMaxScore: 69)

Cartpole solved in 568 runs, with an average score of 21 and a maximum score of 69

Session Summary:
 Total Runs: 568
 Total steps: 8005
 Maximum Score: 69
 Learning Rate: 0.01
 Discount Factor: 0.9
 Exploration Minimum: 0.01
 Exploration Maximum: 0.7
 Exploration Decay: 0.9
 # of Episodes: 20
 Batch Size: 40
 Buffer Size: 2000
This model was successful. Congratulations! Writing to database...
